# Unified LoRA - MRPC Benchmark Example

This notebook demonstrates Unified LoRA on the GLUE MRPC (paraphrase detection) task.

**Expected results:**
- Baseline LoRA: F1 ~0.78-0.79
- Unified LoRA: F1 ~0.78-0.79 (performance parity with adaptive control)
- φ(t) convergence from 0.5 → ~0.35-0.40
- Mode primarily stays in Multi (1) for stable training

## Setup

In [ ]:
!pip install -q transformers datasets peft evaluate scikit-learn accelerate

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
import evaluate

# Import UnifiedController
import sys
sys.path.append('..')
from controller import UnifiedController

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load Data

In [ ]:
dataset = load_dataset("glue", "mrpc")["train"].train_test_split(test_size=0.2, seed=42)
print(f"Dataset: {len(dataset['train'])} train | {len(dataset['test'])} test")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, max_length=128, padding=True)

tokenized_train = dataset['train'].map(tokenize, batched=True).rename_column("label", "labels")
tokenized_test = dataset['test'].map(tokenize, batched=True).rename_column("label", "labels")

metric = evaluate.combine(["accuracy", "f1"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Baseline LoRA

In [ ]:
print("🟡 BASELINE LoRA")
model_base = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model_base = get_peft_model(model_base, LoraConfig(r=16, lora_alpha=32, target_modules=["q_lin", "v_lin"]))

trainer_base = Trainer(
    model=model_base,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=TrainingArguments(
        output_dir="./baseline",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="no",
        report_to=None
    ),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_base.train()
results_base = trainer_base.evaluate()
print(f"F1: {results_base['eval_f1']:.3f} | Acc: {results_base['eval_accuracy']:.3f}")

## Unified LoRA

In [ ]:
print("🔵 UNIFIED LoRA")
controller = UnifiedController()

model_unified = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model_unified = get_peft_model(model_unified, LoraConfig(r=16, lora_alpha=32, target_modules=["q_lin", "v_lin"]))
model_unified = model_unified.to(device)

train_loader = DataLoader(tokenized_train.remove_columns(['sentence1', 'sentence2', 'idx']), batch_size=16, shuffle=True)
optimizer = torch.optim.AdamW(model_unified.parameters(), lr=3e-5)
model_unified.train()

for epoch in range(3):
    print(f"Epoch {epoch+1}/3")
    for i, batch in enumerate(train_loader):
        inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'labels']}
        outputs = model_unified(**inputs)
        new_lr = controller.update(outputs.loss.item())
        for g in optimizer.param_groups: g['lr'] = new_lr
        outputs.loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 50 == 0:
            print(f"  [{controller.step}] φ={controller.phi:.3f} M={controller.mode}")

model_unified.eval()
trainer_unified = Trainer(
    model=model_unified, eval_dataset=tokenized_test,
    args=TrainingArguments(output_dir="./u", per_device_eval_batch_size=16, fp16=True, report_to=None),
    tokenizer=tokenizer, compute_metrics=compute_metrics
)
results_unified = trainer_unified.evaluate()
print(f"F1: {results_unified['eval_f1']:.3f} | Acc: {results_unified['eval_accuracy']:.3f}")

## Results

In [ ]:
print("\n📊 COMPARISON")
print("| Method   | F1    | Acc   |")
print("|----------|-------|-------|")
print(f"| Baseline | {results_base['eval_f1']:.3f} | {results_base['eval_accuracy']:.3f} |")
print(f"| Unified  | {results_unified['eval_f1']:.3f} | {results_unified['eval_accuracy']:.3f} |")
print(f"\nΔF1: {results_unified['eval_f1'] - results_base['eval_f1']:+.3f}")
print(f"Final φ: {controller.phi:.3f} | Mode: {controller.mode}")